In [88]:
import requests
from requests_html import HTML
import pandas as pd
import time

In [6]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = "python"
query_filter = "Votes"
url = f"{base_url}{tag}?tab={query_filter}"
url

'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [7]:
r = requests.get(url)
html_str = r.text
html = HTML(html=html_str)

In [8]:
#html = HTML(html=html_str)

In [9]:
#question-summary is the root elemet of all questions
questions_summaries = html.find(".question-summary")

In [10]:
print(questions_summaries[0].text)


11153
votes
38answers
2.5m views
What does the “yield” keyword do?
What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....
python iterator generator yield coroutine
asked Oct 23 '08 at 22:21
Alex. S.
126k1616 gold badges5050 silver badges6161 bronze badges


In [11]:
columns = ['votes','vote_title','num_answers','views','questions','short_desc','tags','date','user','user_details']

In [15]:
this_row = list(questions_summaries[0].text)
print(this_row)

['1', '1', '1', '5', '3', '\n', 'v', 'o', 't', 'e', 's', '\n', '3', '8', 'a', 'n', 's', 'w', 'e', 'r', 's', '\n', '2', '.', '5', 'm', ' ', 'v', 'i', 'e', 'w', 's', '\n', 'W', 'h', 'a', 't', ' ', 'd', 'o', 'e', 's', ' ', 't', 'h', 'e', ' ', '“', 'y', 'i', 'e', 'l', 'd', '”', ' ', 'k', 'e', 'y', 'w', 'o', 'r', 'd', ' ', 'd', 'o', '?', '\n', 'W', 'h', 'a', 't', ' ', 'i', 's', ' ', 't', 'h', 'e', ' ', 'u', 's', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'y', 'i', 'e', 'l', 'd', ' ', 'k', 'e', 'y', 'w', 'o', 'r', 'd', ' ', 'i', 'n', ' ', 'P', 'y', 't', 'h', 'o', 'n', ',', ' ', 'a', 'n', 'd', ' ', 'w', 'h', 'a', 't', ' ', 'd', 'o', 'e', 's', ' ', 'i', 't', ' ', 'd', 'o', '?', ' ', 'F', 'o', 'r', ' ', 'e', 'x', 'a', 'm', 'p', 'l', 'e', ',', ' ', 'I', "'", 'm', ' ', 't', 'r', 'y', 'i', 'n', 'g', ' ', 't', 'o', ' ', 'u', 'n', 'd', 'e', 'r', 's', 't', 'a', 'n', 'd', ' ', 't', 'h', 'i', 's', ' ', 'c', 'o', 'd', 'e', '1', ':', ' ', 'd', 'e', 'f', ' ', '_', 'g', 'e', 't', '_', 'c', 'h', 'i', 'l',

In [ ]:
#len(this_row)==len(columns)

In [ ]:
#row_data = dict(zip(columns,this_row))

In [ ]:
#for column, row_v in zip(columns,this_row):
#    print(column,this_row)

In [29]:
key_names = ['question','votes','tags']
classes_needed =['.question-hyperlink','.vote','.tags']
this_question_element = questions_summaries[0]
this_question_element.find('.question-hyperlink',first=True).text
this_question_element.find('.vote',first=True).text.replace('\nvotes','')


'11153'

In [51]:
def clean_scraped_data(text,keyname=None):
    if keyname == 'votes':
        return text.replace('\nvotes','')
    return text

In [52]:
datas = []
for q_el in questions_summaries:
    question_data = {}
    for i, _class in enumerate (classes_needed):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text)
    datas.append(question_data)
datas[0]
    

{'question': 'What does the “yield” keyword do?',
 'votes': '11153\nvotes',
 'tags': 'python iterator generator yield coroutine'}

In [74]:
def parse_tagged_page(html):
    questions_summaries = html.find(".question-summary")
    key_names = ['question','votes','tags']
    classes_needed =['.question-hyperlink','.vote','.tags']
    datas = []
    for q_el in questions_summaries:
        question_data = {}
        for i, _class in enumerate (classes_needed):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
        datas.append(question_data)
    return datas

In [81]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [86]:
def scrape_tag(tag = "java", query_filter = "Votes", max_pages=50, pagesize=25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas

In [87]:
scrape_tag()

[{'question': 'Why is processing a sorted array faster than processing an unsorted array?',
  'votes': '25520',
  'tags': 'java c++ performance cpu-architecture branch-prediction'},
 {'question': 'Why Is Subtracting These Two Times (in 1927) Giving A Strange Result?',
  'votes': '7144',
  'tags': 'java date timezone'},
 {'question': 'Is Java “pass-by-reference” or “pass-by-value”?',
  'votes': '7040',
  'tags': 'java methods parameter-passing pass-by-reference pass-by-value'},
 {'question': 'How do I read / convert an InputStream into a String in Java?',
  'votes': '4324',
  'tags': 'java string io stream inputstream'},
 {'question': 'Avoiding NullPointerException in Java',
  'votes': '4188',
  'tags': 'java nullpointerexception null'},
 {'question': 'What are the differences between a HashMap and a Hashtable in Java?',
  'votes': '3959',
  'tags': 'java collections hashmap hashtable'},
 {'question': 'Create ArrayList from array',
  'votes': '3786',
  'tags': 'java arrays arraylist typ

In [89]:
len(datas)

50

In [90]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,What does the “yield” keyword do?,11153\nvotes,python iterator generator yield coroutine
1,What does if __name__ == “__main__”: do?,6802\nvotes,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,6661\nvotes,python operators ternary-operator conditional-...
3,What are metaclasses in Python?,6167\nvotes,python oop metaclass python-class python-datam...
4,How do I check whether a file exists without e...,6135\nvotes,python file file-exists


In [91]:
df.shape

(50, 3)

In [92]:
df.to_csv("java", index=False)